In [1]:
import json

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

### Load data

In [2]:
pdbbind_training_set_pk = pd.read_csv('../data/pdbbind_training_set_binding_affinity.csv', index_col=0, header=None, squeeze=True)

crystal_pose_features = pd.read_csv('../data/crystal_pose_features.csv', index_col=0)
minimised_pose_features = pd.read_csv('../data/minimised_pose_features.csv', index_col=0)
docked_pose_features = pd.read_csv('../data/docked_pose_features.csv', index_col=0)

feature_sets = {}
with open('../data/lb_feature_names.txt') as f:
    feature_sets['LB'] = pd.Index([l.strip() for l in f])
with open('../data/sb_feature_names.txt') as f:
    feature_sets['SB'] = pd.Index([l.strip() for l in f])
with open('../data/hb_feature_names.txt') as f:
    feature_sets['HB'] = pd.Index([l.strip() for l in f])

# We've enumerated the docked poses associated to each PDB structure - these labels are used for cross-validation later
with open('../data/docked_pose_labels.json') as f:
    docked_pose_labels = json.load(f)


### Prepare cross-validation folds

Randomly shuffle and split into five folds - we'll use the same folds across all experiments.

In [3]:
shuffled = pdbbind_training_set_pk.sample(frac=1, replace=False, random_state=42).index
n_test = int(len(shuffled) / 5)
folds = [shuffled[:n_test], shuffled[n_test:2*n_test], shuffled[2*n_test:3*n_test], shuffled[3*n_test:4*n_test], shuffled[4*n_test:]]

pdbbind_training_set = pdbbind_training_set_pk.index

### Cross-validation using crystal poses

For reference, we first perform the cross-validation experiment using crystal poses for training and testing.

In [4]:
cv_crystal_results = {}

for model in feature_sets:
    fold_pearsonr = []
    fold_mse = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        X_train = crystal_pose_features.loc[index_train, feature_sets[model]].values
        X_test = crystal_pose_features.loc[fold, feature_sets[model]].values
        y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()
        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)
        pred = rf.predict(X_test)
        fold_pearsonr.append(stats.pearsonr(y_test, pred)[0])
        fold_mse.append(mean_squared_error(y_test, pred))
    cv_crystal_results[model] = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse))}

with open('../results/cv_crystal_results.json', 'w') as f:
    json.dump(cv_crystal_results, f)

### Experiment 1 - scoring strategy

First we run the cross-validation experiment using different strategies for scoring a ligand when multiple docked poses are available. Three strategies were tested: scoring the pose ranked highest by Smina ("top dock"); scoring all poses and taxing the highest score ("all docks max"); and scoring all poses and taking the mean score ("all docks mean"). Models are trained using a single pose for each ligand, minimised using Smina to achieve a single near-native docked pose. We also train and test using crystal poses for reference.

In [5]:
cv_train_minimized_test_top_dock_results = {}
cv_train_minimized_test_all_docks_max_results = {}
cv_train_minimized_test_all_docks_mean_results = {}

for model in feature_sets:
    fold_pearsonr_max = []
    fold_pearsonr_mean = []
    fold_pearsonr_top = []
    fold_mse_max = []
    fold_mse_mean = []
    fold_mse_top = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        X_train = minimised_pose_features.loc[index_train, feature_sets[model]].values
        y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # List docked poses for this fold
        fold_dock_labels = []
        for pdb in fold:
            fold_dock_labels.extend(docked_pose_labels[pdb])
        fold_dock_labels = pd.Index(fold_dock_labels)

        X_test = docked_pose_features.loc[fold_dock_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_dock_labels)

        # Score all poses, taking max/mean score for each ligand
        max_pred = []
        mean_pred = []

        for pdb in fold:
            max_pred.append(np.max(pred.loc[docked_pose_labels[pdb]]))
            mean_pred.append(np.mean(pred.loc[docked_pose_labels[pdb]]))
        fold_pearsonr_max.append(stats.pearsonr(y_test, max_pred)[0])
        fold_pearsonr_mean.append(stats.pearsonr(y_test, mean_pred)[0])
        fold_mse_max.append(mean_squared_error(y_test, max_pred))
        fold_mse_mean.append(mean_squared_error(y_test, mean_pred))

        # Take the score of the pose ranked highest by Smina
        top_pred = []
        for pdb in fold:
            top_pred.append(pred.loc[docked_pose_labels[pdb][0]])
        fold_pearsonr_top.append(stats.pearsonr(y_test, top_pred)[0])
        fold_mse_top.append(mean_squared_error(y_test, top_pred))
    
    cv_train_minimized_test_all_docks_max_results[model] = {'pearsonr': np.mean(fold_pearsonr_max), 
                                                            'pearsonr_stdev': np.std(fold_pearsonr_max), 
                                                            'rmse': np.sqrt(np.mean(fold_mse_max))}
    
    cv_train_minimized_test_all_docks_mean_results[model] = {'pearsonr': np.mean(fold_pearsonr_mean), 
                                                             'pearsonr_stdev': np.std(fold_pearsonr_mean),
                                                             'rmse': np.sqrt(np.mean(fold_mse_mean))}
    cv_train_minimized_test_top_dock_results[model] = {'pearsonr': np.mean(fold_pearsonr_top), 
                                                       'pearsonr_stdev': np.std(fold_pearsonr_top),
                                                       'rmse': np.sqrt(np.mean(fold_mse_top))}

In [6]:
test_strategy_pearsonr = {
    'Smina top pose': {model: cv_train_minimized_test_top_dock_results[model]['pearsonr'] for model in feature_sets},
    'Maximum pose score': {model: cv_train_minimized_test_all_docks_max_results[model]['pearsonr'] for model in feature_sets},
    'Mean pose score': {model: cv_train_minimized_test_all_docks_mean_results[model]['pearsonr'] for model in feature_sets},
    'Train-test crystal': {m: cv_crystal_results[m]['pearsonr'] for m in feature_sets}
}
test_strategy_pearsonr = pd.DataFrame(test_strategy_pearsonr).loc[['LB','SB','HB']]
test_strategy_pearsonr.index = ['LB model', 'SB model','HB model']
test_strategy_pearsonr.T.to_csv('../results/train_minimised_pose_cv_pearsonr.csv')
test_strategy_pearsonr.T

,LB model,SB model,HB model
Smina top pose,0.718999,0.659097,0.713919
Maximum pose score,0.718999,0.682261,0.732412
Mean pose score,0.718999,0.604533,0.676920
Train-test crystal,0.718999,0.746909,0.767521


### Experiment 2 - Training on docked poses

Next, we repeat the cross-validation experiment, this itme training on the docked pose ranked highest by Smina for each ligand. 

In [7]:
cv_train_top_dock_test_top_dock_results = {}
cv_train_top_dock_test_all_docks_max_results = {}
cv_train_top_dock_test_all_docks_mean_results = {}

for model in feature_sets:
    fold_pearsonr_max = []
    fold_pearsonr_mean = []
    fold_pearsonr_top = []
    fold_mse_max = []
    fold_mse_mean = []
    fold_mse_top = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        training_pose_labels = []

        # Get the labels for the highest-ranked pose for each training complex
        for pdb in index_train:
            training_pose_labels.append(docked_pose_labels[pdb][0])
        training_pose_labels = pd.Index(training_pose_labels)

        X_train = docked_pose_features.loc[training_pose_labels, feature_sets[model]].values
        y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()

        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # List docked poses for this fold
        fold_dock_labels = []
        for pdb in fold:
            fold_dock_labels.extend(docked_pose_labels[pdb])
        fold_dock_labels = pd.Index(fold_dock_labels)

        X_test = docked_pose_features.loc[fold_dock_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_dock_labels)

        # Score all poses, taking max/mean score for each ligand
        max_pred = []
        mean_pred = []

        for pdb in fold:
            max_pred.append(np.max(pred.loc[docked_pose_labels[pdb]]))
            mean_pred.append(np.mean(pred.loc[docked_pose_labels[pdb]]))
        fold_pearsonr_max.append(stats.pearsonr(y_test, max_pred)[0])
        fold_pearsonr_mean.append(stats.pearsonr(y_test, mean_pred)[0])
        fold_mse_max.append(mean_squared_error(y_test, max_pred))
        fold_mse_mean.append(mean_squared_error(y_test, mean_pred))

        # Take the score of the pose ranked highest by Smina
        top_pred = []
        for pdb in fold:
            top_pred.append(pred.loc[docked_pose_labels[pdb][0]])
        fold_pearsonr_top.append(stats.pearsonr(y_test, top_pred)[0])
        fold_mse_top.append(mean_squared_error(y_test, top_pred))

    cv_train_top_dock_test_all_docks_max_results[model] = {'pearsonr': np.mean(fold_pearsonr_max), 
                                                            'pearsonr_stdev': np.std(fold_pearsonr_max), 
                                                            'rmse': np.sqrt(np.mean(fold_mse_max))}
    
    cv_train_top_dock_test_all_docks_mean_results[model] = {'pearsonr': np.mean(fold_pearsonr_mean), 
                                                            'pearsonr_stdev': np.std(fold_pearsonr_mean), 
                                                            'rmse': np.sqrt(np.mean(fold_mse_mean))}
    cv_train_top_dock_test_top_dock_results[model] = {'pearsonr': np.mean(fold_pearsonr_top), 
                                                      'pearsonr_stdev': np.std(fold_pearsonr_top), 
                                                      'rmse': np.sqrt(np.mean(fold_mse_top))}

In [8]:
test_strategy_pearsonr = {
    'Smina top pose': {model: cv_train_top_dock_test_top_dock_results[model]['pearsonr'] for model in feature_sets},
    'Maximum pose score': {model: cv_train_top_dock_test_all_docks_max_results[model]['pearsonr'] for model in feature_sets},
    'Mean pose score': {model: cv_train_top_dock_test_all_docks_mean_results[model]['pearsonr'] for model in feature_sets},
    'Train-test crystal': {m: cv_crystal_results[m]['pearsonr'] for m in feature_sets}
}
test_strategy_pearsonr = pd.DataFrame(test_strategy_pearsonr).loc[['LB','SB','HB']]
test_strategy_pearsonr.index = ['LB model', 'SB model','HB model']
test_strategy_pearsonr.T.to_csv('../results/train_top_docked_pose_cv_pearsonr.csv')
test_strategy_pearsonr.T

,LB model,SB model,HB model
Smina top pose,0.718999,0.676220,0.738276
Maximum pose score,0.718999,0.687132,0.744309
Mean pose score,0.718999,0.643699,0.725241
Train-test crystal,0.718999,0.746909,0.767521


### Experiment 3 - Training using multiple poses

Next we again repeat the cross-validation experiment, this time training on all of the docked poses for each ligand. To control for the effect of increasing the size of training set, we also repeat the experiment by training on a number of redundant copies of the top pose for each ligand equal to the number of docked poses.

In [12]:
# run cv on docks, training on all docks
cv_train_all_docks_test_all_docks_results = {}

for model in feature_sets:
    fold_pearsonr = []
    fold_mse = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        
        training_pose_labels = []
        for pdb in index_train:
            training_pose_labels.extend(docked_pose_labels[pdb])
        training_pose_labels = pd.Index(training_pose_labels)

        X_train = docked_pose_features.loc[training_pose_labels, feature_sets[model]].values
        # Training affinities are the same for each pose of a ligand
        training_pose_pdbs = pd.Index(i[:4] for i in training_pose_labels)
        y_train = pdbbind_training_set_pk.loc[training_pose_pdbs].values.ravel()

        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # test on all docks
        fold_pose_labels = []
        for pdb in fold:
            fold_pose_labels.extend(docked_pose_labels[pdb])
        fold_pose_labels = pd.Index(fold_pose_labels)
        X_test = docked_pose_features.loc[fold_pose_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_pose_labels)
        max_pred = []
        for pdb in fold:
            max_pred.append(np.max(pred.loc[docked_pose_labels[pdb]]))
        fold_pearsonr.append(stats.pearsonr(y_test, max_pred)[0])
        fold_mse.append(mean_squared_error(y_test, max_pred))

    cv_train_all_docks_test_all_docks_results[model] = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse))}

In [13]:
cv_train_redundant_docks_test_all_docks_results = {}

for model in feature_sets:
    fold_pearsonr = []
    fold_mse = []
    for fold in folds:

        index_train = pdbbind_training_set.difference(fold)
        
        training_pose_labels = []
        for pdb in index_train:
            training_pose_labels.extend(docked_pose_labels[pdb])
        training_pose_labels = pd.Index(training_pose_labels)

        # This time we want N copies of the pose ranked highest by Smina
        training_pose_labels = training_pose_labels.map(lambda x: x[:4]+'_0')
        X_train = docked_pose_features.loc[training_pose_labels, feature_sets[model]].values

        # Training affinities are the same for each pose of a ligand
        training_pose_pdbs = pd.Index(i[:4] for i in training_pose_labels)
        y_train = pdbbind_training_set_pk.loc[training_pose_pdbs].values.ravel()

        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # test on all docks
        fold_pose_labels = []
        for pdb in fold:
            fold_pose_labels.extend(docked_pose_labels[pdb])
        fold_pose_labels = pd.Index(fold_pose_labels)
        X_test = docked_pose_features.loc[fold_pose_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_pose_labels)
        max_pred = []
        for pdb in fold:
            max_pred.append(np.max(pred.loc[docked_pose_labels[pdb]]))
        fold_pearsonr.append(stats.pearsonr(y_test, max_pred)[0])
        fold_mse.append(mean_squared_error(y_test, max_pred))
    
    cv_train_redundant_docks_test_all_docks_results[model] = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse))}

In [14]:
multipose_pearsonr = {}

for m in feature_sets:
    multipose_pearsonr[m] = {'Smina top pose': cv_train_top_dock_test_all_docks_max_results[m]['pearsonr'],
                                'All poses': cv_train_all_docks_test_all_docks_results[m]['pearsonr'],
                                'Redundant poses': cv_train_redundant_docks_test_all_docks_results[m]['pearsonr']}

multipose_pearsonr = pd.DataFrame(multipose_pearsonr)[['LB', 'SB', 'HB']]
multipose_pearsonr.columns = ['LB model', 'SB model','HB model']
multipose_pearsonr.to_csv('../results/train_multiple_poses_cv_pearsonr.csv')
multipose_pearsonr



,LB model,SB model,HB model
Smina top pose,0.718999,0.687132,0.744309
All poses,0.712263,0.699169,0.723782
Redundant poses,0.712263,0.687649,0.744613
